In [37]:
import pandas as pd
import numpy as np
data=pd.read_excel("/content/LIHC_SampleStageFirst_GeneExpression_HGNC.xlsx")

In [3]:
data.head()

,Sample_short,Sample_ID_full,Stage,Patient_ID,Diagnosis_Age,TSPAN6,TNMD,DPM1,SCYL3,FIRRM,...,FPR3,H1-6,C11orf96,HSD3BP1,SERPINA13P,KCNJ11,CDHR4,COL4A1,RPL7P48,PLEKHG7
0,TCGA-RC-A7SB-01,TCGA-RC-A7SB-01A,Stage II,TCGA-RC-A7SB,53.0,9646,4,2507,383,284,...,172,1,131,4,0,164,2,6665,0,21
1,TCGA-CC-A7IE-01,TCGA-CC-A7IE-01A,Stage IIIA,TCGA-CC-A7IE,57.0,5279,0,1445,360,202,...,243,1,525,0,5,37,8,10742,3,12
2,TCGA-RC-A6M4-01,TCGA-RC-A6M4-01A,Stage IIIA,TCGA-RC-A6M4,74.0,6951,0,1280,483,215,...,72,0,94,1,3,54,2,3998,7,5
3,TCGA-CC-A3MB-01,TCGA-CC-A3MB-01A,Stage IIIA,TCGA-CC-A3MB,36.0,6683,0,2351,1158,749,...,381,0,204,1,0,72,3,13194,2,3
4,TCGA-HP-A5MZ-01,TCGA-HP-A5MZ-01A,Stage I,TCGA-HP-A5MZ,62.0,10285,2,1131,577,188,...,1115,1,727,0,0,101,7,13616,0,19


In [4]:
data.shape

(374, 16384)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374 entries, 0 to 373
Columns: 16384 entries, Sample_short to PLEKHG7
dtypes: float64(1), int64(16379), object(4)
memory usage: 46.8+ MB


In [38]:
if data.duplicated().sum() > 0:
    data = data.drop_duplicates() #ensure no redundancy

In [39]:
og_len= len(data)
data = data.dropna(subset=['Stage'])
print("samples w/ missing stage and dropped", og_len-len(data)) #cant use samples w/ no stage"

samples w/ missing stage and dropped 24


In [40]:
irrelevant_col= ['Sample_short','Sample_ID_full','Patient_ID', 'Diagnosis_Age']
genes_col = [col for col in data.columns if col not in irrelevant_col and col != 'Stage']
print("genes number",len(genes_col)) #we only need genes for training

genes number 16379


In [41]:
missing=data[genes_col].isnull().mean()
drop_gene= missing[missing>0.30].index
data = data.drop(columns=drop_gene)
genes_col =[col for col in genes_col if col not in drop_gene]
data[genes_col] = data[genes_col].fillna(data[genes_col].median()) #exclude genes w/ high % missing, impute the low % w/ median
print("Remaining genes,missing dropped", len(genes_col))

Remaining genes,missing dropped 16379


In [44]:
zeros=(data[genes_col]==0).mean()
drop_gene0 = zeros[zeros>0.90].index
data = data.drop(columns=drop_gene0)
genes_col = [col for col in genes_col if col not in drop_gene0]
print(len(drop_gene0),"genes dropped,zero variance")
print(len(genes_col),"genes remain") #exclude genes w/ zero

0 genes dropped,zero variance
15722 genes remain


In [7]:
from sklearn.preprocessing import LabelEncoder, QuantileTransformer, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import f_classif
from statsmodels.stats.multitest import multipletests
from imblearn.over_sampling import SMOTE
from collections import Counter

In [26]:
cpm=data[genes_col].sum(axis=1)
data[genes_col]=data[genes_col] .div(cpm,axis=0) * 1e6 #apply cpm for within sample normalization

In [64]:
data['Main_Stage'] = data['Stage'].str.extract(r'Stage\s+([IVX]+)', expand=False)
data['Main_Stage'] = 'Stage ' + data['Main_Stage']
data['Main_Stage'] = data['Main_Stage'].replace('Stage IV', 'Stage III')
print(data['Main_Stage'].value_counts())

Main_Stage
Stage I      173
Stage III     90
Stage II      87
Name: count, dtype: int64


In [68]:
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import confusion_matrix, classification_report
smote = SMOTE(random_state=42, k_neighbors=5)
X_train_res, y_train_res = smote.fit_resample(X_train_scaled, y_train)

models = {
    "LogisticRegression": LogisticRegression(
        max_iter=5000,
        multi_class="multinomial",
        penalty='elasticnet',
        solver='saga',
        l1_ratio=0.5
    ),
    "LinearSVM": LinearSVC(
        penalty='l2',
        max_iter=5000
    ),
    "RBF_SVM": SVC(
        kernel="rbf",
        probability=True
    ),
    "RandomForest": RandomForestClassifier(
        n_estimators=300,
        random_state=42,
        n_jobs=-1
    ),
    "GradientBoosting": GradientBoostingClassifier()
}

for name, clf in models.items():
    print("\n" + "="*30)
    print(f"Model: {name}")

    clf.fit(X_train_res, y_train_res)

    y_pred = clf.predict(X_test_scaled)

    print("Confusion matrix:")
    print(confusion_matrix(y_test, y_pred))

    print("\nClassification report:")
    print(classification_report(y_test, y_pred, target_names=le.classes_))


Model: LogisticRegression


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Confusion matrix:
[[25  5  5]
 [ 8  5  4]
 [ 4  5  9]]

Classification report:
              precision    recall  f1-score   support

     Stage I       0.68      0.71      0.69        35
    Stage II       0.33      0.29      0.31        17
   Stage III       0.50      0.50      0.50        18

    accuracy                           0.56        70
   macro avg       0.50      0.50      0.50        70
weighted avg       0.55      0.56      0.55        70


Model: LinearSVM


/usr/local/lib/python3.12/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Confusion matrix:
[[26  4  5]
 [ 7  4  6]
 [ 5  5  8]]

Classification report:
              precision    recall  f1-score   support

     Stage I       0.68      0.74      0.71        35
    Stage II       0.31      0.24      0.27        17
   Stage III       0.42      0.44      0.43        18

    accuracy                           0.54        70
   macro avg       0.47      0.47      0.47        70
weighted avg       0.53      0.54      0.53        70


Model: RBF_SVM
Confusion matrix:
[[26  2  7]
 [ 6  1 10]
 [ 7  3  8]]

Classification report:
              precision    recall  f1-score   support

     Stage I       0.67      0.74      0.70        35
    Stage II       0.17      0.06      0.09        17
   Stage III       0.32      0.44      0.37        18

    accuracy                           0.50        70
   macro avg       0.38      0.42      0.39        70
weighted avg       0.46      0.50      0.47        70


Model: RandomForest
Confusion matrix:
[[27  2  6]
 [ 9  2  6]
 

In [50]:
from imblearn.over_sampling import BorderlineSMOTE
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import confusion_matrix

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Borderline-SMOTE
smote = BorderlineSMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

models = {
    "LogisticRegression": LogisticRegression(
        max_iter=5000,
        multi_class="multinomial"
    ),
    "LinearSVM": LinearSVC(),
    "RBF_SVM": SVC(
        kernel="rbf",
        probability=False
    ),
    "RandomForest": RandomForestClassifier(
        n_estimators=300,
        random_state=42,
        n_jobs=-1
    ),
    "GradientBoosting": GradientBoostingClassifier()
}

for name, clf in models.items():
    print("\n==============================")
    print(f"Model: {name}")
    clf.fit(X_train_res, y_train_res)
    y_pred = clf.predict(X_test)

    print("Confusion matrix:")
    print(confusion_matrix(y_test, y_pred))

    print("\nClassification report:")
    print(classification_report(y_test, y_pred))



Model: LogisticRegression


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Confusion matrix:
[[17 18  0]
 [11  6  0]
 [ 1  0  0]]

Classification report:
              precision    recall  f1-score   support

           0       0.59      0.49      0.53        35
           1       0.25      0.35      0.29        17
           2       0.00      0.00      0.00         1

    accuracy                           0.43        53
   macro avg       0.28      0.28      0.27        53
weighted avg       0.47      0.43      0.44        53


Model: LinearSVM


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/s

Confusion matrix:
[[22 13  0]
 [13  4  0]
 [ 1  0  0]]

Classification report:
              precision    recall  f1-score   support

           0       0.61      0.63      0.62        35
           1       0.24      0.24      0.24        17
           2       0.00      0.00      0.00         1

    accuracy                           0.49        53
   macro avg       0.28      0.29      0.29        53
weighted avg       0.48      0.49      0.48        53


Model: RBF_SVM


...

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Confusion matrix:
[[27  8  0]
 [ 9  8  0]
 [ 1  0  0]]

Classification report:
              precision    recall  f1-score   support

           0       0.73      0.77      0.75        35
           1       0.50      0.47      0.48        17
           2       0.00      0.00      0.00         1

    accuracy                           0.66        53
   macro avg       0.41      0.41      0.41        53
weighted avg       0.64      0.66      0.65        53


Model: GradientBoosting


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Confusion matrix:
[[26  9  0]
 [ 9  8  0]
 [ 1  0  0]]

Classification report:
              precision    recall  f1-score   support

           0       0.72      0.74      0.73        35
           1       0.47      0.47      0.47        17
           2       0.00      0.00      0.00         1

    accuracy                           0.64        53
   macro avg       0.40      0.40      0.40        53
weighted avg       0.63      0.64      0.63        53



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [58]:
from imblearn.over_sampling import ADASYN

adasyn = ADASYN(random_state=42, n_neighbors=1)
X_train_res, y_train_res = adasyn.fit_resample(X_train, y_train)


print(classification_report(y_test, y_pred))

models = {
    "LogisticRegression": LogisticRegression(
        max_iter=5000,
        multi_class="multinomial"
    ),
    "LinearSVM": LinearSVC(),
    "RBF_SVM": SVC(
        kernel="rbf",
        probability=False
    ),
    "RandomForest": RandomForestClassifier(
        n_estimators=300,
        random_state=42,
        n_jobs=-1
    ),
    "GradientBoosting": GradientBoostingClassifier()
}

for name, clf in models.items():
    print("\n==============================")
    print(f"Model: {name}")
    clf.fit(X_train_res, y_train_res)
    y_pred = clf.predict(X_test)

    print("Confusion matrix:")
    print(confusion_matrix(y_test, y_pred))

    print("\nClassification report:")
    print(classification_report(y_test, y_pred))


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

           0       0.68      0.60      0.64        35
           1       0.36      0.47      0.41        17
           2       0.00      0.00      0.00         1

    accuracy                           0.55        53
   macro avg       0.35      0.36      0.35        53
weighted avg       0.56      0.55      0.55        53


Model: LogisticRegression


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Confusion matrix:
[[17 18  0]
 [11  6  0]
 [ 0  1  0]]

Classification report:
              precision    recall  f1-score   support

           0       0.61      0.49      0.54        35
           1       0.24      0.35      0.29        17
           2       0.00      0.00      0.00         1

    accuracy                           0.43        53
   macro avg       0.28      0.28      0.28        53
weighted avg       0.48      0.43      0.45        53


Model: LinearSVM


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/s

...

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Confusion matrix:
[[29  6  0]
 [11  6  0]
 [ 1  0  0]]

Classification report:
              precision    recall  f1-score   support

           0       0.71      0.83      0.76        35
           1       0.50      0.35      0.41        17
           2       0.00      0.00      0.00         1

    accuracy                           0.66        53
   macro avg       0.40      0.39      0.39        53
weighted avg       0.63      0.66      0.64        53


Model: GradientBoosting


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Confusion matrix:
[[27  8  0]
 [12  5  0]
 [ 1  0  0]]

Classification report:
              precision    recall  f1-score   support

           0       0.68      0.77      0.72        35
           1       0.38      0.29      0.33        17
           2       0.00      0.00      0.00         1

    accuracy                           0.60        53
   macro avg       0.35      0.36      0.35        53
weighted avg       0.57      0.60      0.58        53



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [53]:
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SMOTE

custom_smote = SMOTE(k_neighbors=1, random_state=42)

smote_enn = SMOTEENN(smote=custom_smote, random_state=42)
X_train_res, y_train_res = smote_enn.fit_resample(X_train, y_train)

models = {
    "LogisticRegression": LogisticRegression(
        max_iter=5000,
        multi_class="multinomial"
    ),
    "LinearSVM": LinearSVC(),
    "RBF_SVM": SVC(
        kernel="rbf",
        probability=False
    ),
    "RandomForest": RandomForestClassifier(
        n_estimators=300,
        random_state=42,
        n_jobs=-1
    ),
    "GradientBoosting": GradientBoostingClassifier()
}

for name, clf in models.items():
    print("\n==============================")
    print(f"Model: {name}")
    clf.fit(X_train_res, y_train_res)
    y_pred = clf.predict(X_test)

    print("Confusion matrix:")
    print(confusion_matrix(y_test, y_pred))

    print("\nClassification report:")
    print(classification_report(y_test, y_pred))



Model: LogisticRegression


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Confusion matrix:
[[19 15  1]
 [ 6 10  1]
 [ 1  0  0]]

Classification report:
              precision    recall  f1-score   support

           0       0.73      0.54      0.62        35
           1       0.40      0.59      0.48        17
           2       0.00      0.00      0.00         1

    accuracy                           0.55        53
   macro avg       0.38      0.38      0.37        53
weighted avg       0.61      0.55      0.56        53


Model: LinearSVM


/usr/local/lib/python3.12/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Confusion matrix:
[[19 16  0]
 [ 8  9  0]
 [ 1  0  0]]

Classification report:
              precision    recall  f1-score   support

           0       0.68      0.54      0.60        35
           1       0.36      0.53      0.43        17
           2       0.00      0.00      0.00         1

    accuracy                           0.53        53
   macro avg       0.35      0.36      0.34        53
weighted avg       0.56      0.53      0.54        53


Model: RBF_SVM


...

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Confusion matrix:
[[13 22  0]
 [ 3 14  0]
 [ 1  0  0]]

Classification report:
              precision    recall  f1-score   support

           0       0.76      0.37      0.50        35
           1       0.39      0.82      0.53        17
           2       0.00      0.00      0.00         1

    accuracy                           0.51        53
   macro avg       0.38      0.40      0.34        53
weighted avg       0.63      0.51      0.50        53


Model: GradientBoosting


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Confusion matrix:
[[13 22  0]
 [ 5 12  0]
 [ 1  0  0]]

Classification report:
              precision    recall  f1-score   support

           0       0.68      0.37      0.48        35
           1       0.35      0.71      0.47        17
           2       0.00      0.00      0.00         1

    accuracy                           0.47        53
   macro avg       0.35      0.36      0.32        53
weighted avg       0.57      0.47      0.47        53



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [56]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif

selector = SelectKBest(f_classif, k=100)
X_train_selected_scaled = selector.fit_transform(X_train_scaled, y_train)
X_test_selected_scaled = selector.transform(X_test_scaled)

smote = SMOTE(random_state=42, k_neighbors=1)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_selected_scaled, y_train)


elastic_net = LogisticRegression(
    penalty='elasticnet',
    solver='saga',
    l1_ratio=0.5,
    C=1.0,
    max_iter=1000,
    random_state=42,
    n_jobs=-1
)

elastic_net.fit(X_train_resampled, y_train_resampled)
print(f"elastic_net Accuracy: {elastic_net.score(X_test_selected_scaled, y_test):.4f}")
print(f"\nCoefficients shape: {elastic_net.coef_.shape}")
print(f"Intercept: {elastic_net.intercept_}")

y_pred = elastic_net.predict(X_test_selected_scaled)
print("\nClassification Report:")
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

print(f"\nPrediction for first test sample:")
print(f"Predicted class: {elastic_net.predict(X_test_selected_scaled[[0]])}")
print(f"Prediction probabilities: {elastic_net.predict_proba(X_test_selected_scaled[[0]])}")

elastic_net Accuracy: 0.5472

Coefficients shape: (3, 100)
Intercept: [ 1.7725852   0.68768865 -2.46027385]

Classification Report:
              precision    recall  f1-score   support

           0       0.68      0.60      0.64        35
           1       0.36      0.47      0.41        17
           2       0.00      0.00      0.00         1

    accuracy                           0.55        53
   macro avg       0.35      0.36      0.35        53
weighted avg       0.56      0.55      0.55        53


Prediction for first test sample:
Predicted class: [1]
Prediction probabilities: [[0.48882536 0.50657763 0.00459701]]


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE, SelectKBest, f_classif
from sklearn.pipeline import Pipeline
pre_selector = SelectKBest(f_classif, k=1000)
X_train_filtered = pre_selector.fit_transform(X_train_resampled, y_train_resampled)
base_model = LogisticRegression(
    max_iter=1000,
    solver='saga',
    multi_class='multinomial',
    n_jobs=-1
)

selector = RFE(estimator=base_model, n_features_to_select=50, step=0.1)
selector = selector.fit(X_train_filtered, y_train_resampled)
mask_1 = pre_selector.get_support()
mask_2 = selector.support_
all_genes = X.columns[mask_1]
final_selected_genes = all_genes[mask_2]

print(f"Selected {len(final_selected_genes)} genes)

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: 

Selected 50 genes in a fraction of the time!


In [19]:
from sklearn.metrics import classification_report, accuracy_score
pre_selected_genes = X.columns[pre_selector.get_support()]
final_genes = pre_selected_genes[selector.support_]

print(f"--- Selected Genes ({len(final_genes)}) ---")
print(final_genes.tolist())

X_test_filtered = pre_selector.transform(X_test_scaled)
X_test_final = selector.transform(X_test_filtered)

y_pred = selector.predict(X_test_filtered)
accuracy = accuracy_score(y_test, y_pred)

print(f"\n--- Model Performance ---")
print(f"Final Test Accuracy: {accuracy:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=le.classes_))

--- Selected Genes (50) ---
['SNAI2', 'NRXN3', 'ACSM2B', 'PRKCZ', 'EPHA6', 'ERGIC2', 'OSGEP', 'SUSD2', 'UPB1', 'CBLN1', 'CLUAP1', 'BLVRA', 'ABCA2', 'MANBA', 'DAP', 'PAPPA2', 'ADGRL2', 'DCAF4', 'ACAT2', 'WDR24', 'STYXL1', 'MRPS25', 'FKBP11', 'ACTR1A', 'TDG', 'SNTG1', 'HNMT', 'ZNF385D', 'GPR180', 'CMTM7', 'DNAAF1', 'RALGDS', 'SLC2A6', 'TMEM82', 'SLC29A4', 'CHMP4C', 'GLYCTK', 'KRT83', 'SLC26A5', 'THAP2', 'CA5A', 'SMIM19', 'AQP11', 'WASH6P', 'ACSM2A', 'H2BC21', 'EFCAB10', 'SRL', 'TMEM210', 'ENTPD5']

--- Model Performance ---
Final Test Accuracy: 0.4906

Classification Report:
              precision    recall  f1-score   support

     Stage I       0.67      0.51      0.58        35
    Stage II       0.31      0.47      0.37        17
   Stage III       0.00      0.00      0.00         1

    accuracy                           0.49        53
   macro avg       0.32      0.33      0.32        53
weighted avg       0.54      0.49      0.50        53



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
f_val, p_val = f_classif(X_train_balanced, y_train_balanced)
fdr = multipletests(p_val, alpha=0.01, method='fdr_bh')[1]
means = X_train_balanced.groupby(y_train_balanced).mean()
fold_change = means.max() / (means.min())
passing = (fdr < 0.01) & (p_val < 0.05) & (fold_change > 3) #selects most signif genes
sig_genes= X_train_balanced.columns[passing]
X_train_anova = X_train_balanced[sig_genes]
X_test_anova = X_test[sig_genes]
print(f"Genes passing FDR<0.01,p<0.05 FC>3: {len(sig_genes)}") #from lit

Genes passing FDR<0.01,p<0.05 FC>: 10186


In [ ]:
import matplotlib.pyplot as plt
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
classifiers = {
    "RandomForest": RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1),
    "SVM_Linear": SVC(kernel="linear", random_state=42),
    "AdaBoost": AdaBoostClassifier(n_estimators=50, random_state=42)
}
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42) #10 fold
best_acc, best_name, best_genes = 0, "", []
for name, model in classifiers.items():
    selector = RFECV(estimator=model, step=1, cv=cv, scoring='accuracy', min_features_to_select=4, n_jobs=-1)
    selector.fit(X_train_scaled, y_train_balanced)
    acc = selector.cv_results_['mean_test_score'].max()
    print(f"{name}: Acc={acc:.4f}, Genes={selector.n_features_}")
    if acc > best_acc:
        best_acc = acc
        best_name = name
        best_genes = X_train_scaled.columns[selector.support_].tolist()
print(f"best {best_name} ({best_acc})")
X_train_wrapper = X_train_scaled[best_genes]
X_test_wrapper = X_test_scaled[best_genes]
print(f"Final Feature subset {len(best_genes)}")

RandomForest: Acc=0.9597, Genes=2762
SVM_Linear: Acc=0.9483, Genes=802
